In [67]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
df.dropna(inplace=True)

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Download necessary NLTK data
nltk.download('vader_lexicon')
nltk.download('punkt')

# Load the CSV file with specified encoding
file_path = r"C:\Users\Home\Downloads\test (3).csv"
df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Convert all values in the text column to strings and handle missing values
df['text'] = df['text'].astype(str).fillna('')

# Display the DataFrame
df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265.0,470.0,164.0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0


In [69]:
df.dropna(inplace=True)

In [71]:
# Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df['text'] = df['text'].apply(preprocess_text)

In [73]:
df['text'].head(10)

0                 last session day httptwitpiccom67ezh
1    shanghai also really exciting precisely skyscr...
2    recession hit veronique branquinho quit compan...
3                                           happy bday
4                             httptwitpiccom4w75p like
5                             thats great weee visitor
6                              think everyone hate lol
7    soooooo wish could im school myspace completel...
8                          within short time last clue
9    get day alright havent done anything yet leavi...
Name: text, dtype: object

In [75]:
df.head(10)

,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,f87dea47db,last session day httptwitpiccom67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,96d74cb729,shanghai also really exciting precisely skyscr...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0
2,eee518ae67,recession hit veronique branquinho quit compan...,negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,01082688c6,happy bday,positive,morning,46-60,Andorra,77265.0,470.0,164.0
4,33987a8ee5,httptwitpiccom4w75p like,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0
5,726e501993,thats great weee visitor,positive,night,70-100,Antigua and Barbuda,97929.0,440.0,223.0
6,261932614e,think everyone hate lol,negative,morning,0-20,Argentina,45195774.0,2736690.0,17.0
7,afa11da83f,soooooo wish could im school myspace completel...,negative,noon,21-30,Armenia,2963243.0,28470.0,104.0
8,e64208b4ef,within short time last clue,neutral,night,31-45,Australia,25499884.0,7682300.0,3.0
9,37bcad24ca,get day alright havent done anything yet leavi...,neutral,morning,46-60,Austria,9006398.0,82400.0,109.0


In [77]:
# Initialize the VADER sentiment intensity analyzer
sid = SentimentIntensityAnalyzer()

# Function to get the sentiment score
def get_sentiment_score(text):
    scores = sid.polarity_scores(text)
    return scores

# Apply the function to the DataFrame
df['sentiment_scores'] = df['text'].apply(get_sentiment_score)

# Function to determine final sentiment based on compound score
def final_sentiment(compound):
    if compound >= 0.10:
        return 'positive'
    elif compound <= -0.10:
        return 'negative'
    else:
        return 'neutral'

# Apply the final sentiment function to the DataFrame
df['final_sentiment'] = df['sentiment_scores'].apply(lambda x: final_sentiment(x['compound']))

# Display the DataFrame with sentiment scores and final sentiment
df.head()

,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²),sentiment_scores,final_sentiment
0,f87dea47db,last session day httptwitpiccom67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral
1,96d74cb729,shanghai also really exciting precisely skyscr...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0,"{'neg': 0.0, 'neu': 0.61, 'pos': 0.39, 'compou...",positive
2,eee518ae67,recession hit veronique branquinho quit compan...,negative,night,31-45,Algeria,43851044.0,2381740.0,18.0,"{'neg': 0.541, 'neu': 0.459, 'pos': 0.0, 'comp...",negative
3,01082688c6,happy bday,positive,morning,46-60,Andorra,77265.0,470.0,164.0,"{'neg': 0.0, 'neu': 0.213, 'pos': 0.787, 'comp...",positive
4,33987a8ee5,httptwitpiccom4w75p like,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0,"{'neg': 0.0, 'neu': 0.286, 'pos': 0.714, 'comp...",positive


In [79]:
from sklearn.metrics import classification_report
 
# Ensure columns contain only strings
df['sentiment'] = df['sentiment'].astype(str)
df['final_sentiment'] = df['final_sentiment'].astype(str)
 
# Print the classification report for VADER
print("\nVADER Classification Report:")
print(classification_report(df['sentiment'], df['final_sentiment']))


VADER Classification Report:
              precision    recall  f1-score   support

    negative       0.71      0.57      0.63      1001
     neutral       0.69      0.51      0.59      1430
    positive       0.58      0.87      0.69      1103

    accuracy                           0.64      3534
   macro avg       0.66      0.65      0.64      3534
weighted avg       0.66      0.64      0.63      3534



In [81]:
# Preprocess text data
def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    return tokens

df['tokens'] = df['text'].apply(preprocess_text)

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Vectorize text data using Word2Vec
def vectorize_text(tokens, model):
    vector = np.zeros(model.vector_size)
    count = 0
    for token in tokens:
        if token in model.wv.key_to_index:
            vector += model.wv[token]
            count += 1
    if count != 0:
        vector /= count
    return vector

df['vector'] = df['tokens'].apply(lambda x: vectorize_text(x, word2vec_model))

In [83]:
df['vector']

0       [-0.013849512324668467, 0.014759456273168325, ...
1       [-0.004895619834618022, 0.0033858509462637207,...
2       [-6.5052615744727e-05, 0.002306538534217647, -...
3       [0.0029051120509393513, 0.010322751011699438, ...
4       [-0.015536025632172823, 0.023985980813449714, ...
                              ...                        
3529    [-0.009542831918224692, 0.028368771076202393, ...
3530    [-0.004603378800675273, 0.0075027830355490245,...
3531    [-0.00375081836245954, 0.01452718215296045, 0....
3532    [-0.003884290374116972, 0.012978789542103186, ...
3533    [-0.0017759899298350017, 0.001633494236102706,...
Name: vector, Length: 3534, dtype: object

In [85]:
# Prepare data for machine learning models
X = np.array(df['vector'].tolist())
y = df['sentiment']

# Encode target labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [87]:
# Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)

# Support Vector Machine
svm = SVC()
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

# Naive Bayes
# Naive Bayes typically works with TF-IDF vectors rather than Word2Vec
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(df['text'])

X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, y_encoded, test_size=0.2, random_state=42)
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train_tfidf)
y_pred_nb = nb.predict(X_test_tfidf)

# Print classification reports
print("\nLogistic Regression Classification Report:")
print(classification_report(y_test, y_pred_log_reg, target_names=le.classes_))

print("\nSVM Classification Report:")
print(classification_report(y_test, y_pred_svm, target_names=le.classes_))

print("\nNaive Bayes Classification Report:")
print(classification_report(y_test_tfidf, y_pred_nb, target_names=le.classes_))


Logistic Regression Classification Report:
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00       207
     neutral       0.42      0.95      0.58       286
    positive       0.50      0.12      0.19       214

    accuracy                           0.42       707
   macro avg       0.31      0.36      0.26       707
weighted avg       0.32      0.42      0.29       707


SVM Classification Report:
              precision    recall  f1-score   support

    negative       0.78      0.03      0.06       207
     neutral       0.45      0.95      0.61       286
    positive       0.73      0.30      0.42       214

    accuracy                           0.49       707
   macro avg       0.65      0.43      0.37       707
weighted avg       0.63      0.49      0.39       707


Naive Bayes Classification Report:
              precision    recall  f1-score   support

    negative       0.76      0.32      0.45       207
     neutral       0.

C:\Users\Home\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Home\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Home\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [89]:
text = "I really really like the song Love Story by Taylor Swift"
scores = sid.polarity_scores(text)

In [91]:
text = "this is waste of product"
scores = sid.polarity_scores(text)

In [93]:
text = "bootcamp is excellent"
scores = sid.polarity_scores(text)
scores

{'neg': 0.0, 'neu': 0.351, 'pos': 0.649, 'compound': 0.5719}

In [95]:
text = "bootcamp is not bad "
scores = sid.polarity_scores(text)
scores

{'neg': 0.0, 'neu': 0.513, 'pos': 0.487, 'compound': 0.431}

In [97]:
text = "bootcamp is not good "
scores = sid.polarity_scores(text)
scores

{'neg': 0.445, 'neu': 0.555, 'pos': 0.0, 'compound': -0.3412}

In [99]:
text = "bootcamp is somewhat good somewhat bad"
scores = sid.polarity_scores(text)
scores

{'neg': 0.308, 'neu': 0.419, 'pos': 0.273, 'compound': -0.0865}

In [101]:
text = "bootcamp is too good.I like bootcamp a lot"
scores = sid.polarity_scores(text)

In [103]:
text = "bootcamp is too bad"
scores = sid.polarity_scores(text)
scores

{'neg': 0.538, 'neu': 0.462, 'pos': 0.0, 'compound': -0.5423}

In [105]:
text = "bootcamp is too good"
scores = sid.polarity_scores(text)
scores

{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}

In [107]:
scores

{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}

In [109]:
def final_sentiment(compound):
    if compound >= 0.10:
        return 'positive'
    elif compound <= -0.10:
        return 'negative'
    else:
        return 'neutral'